In [ ]:
from rasa_nlu.model import Interpreter
import json

In [ ]:
# nlu_config = """
# language: fr
# pipeline: tensorflow_embedding
# """

nlu_config = """
language: fr
pipeline:
- name: "nlp_spacy"
- name: "tokenizer_spacy"
- name: "intent_featurizer_spacy"
- name: "intent_entity_featurizer_regex"
- name: "ner_crf"
- name: "ner_synonyms"
- name: "intent_classifier_sklearn"
"""

# nlu_config = """
# language: fr
# pipeline: "spacy_sklearn"
# """

%store nlu_config > nlu_config.yml

In [ ]:
nlu_md = """
## intent:hello
- salut
- bonjour
- hey
- hello
- coucou
- yo

## intent:search_dataset
- statistiques de [la ville de paris](organization)
- je cherche [les données](object) du [ministère de l'intérieur](organization)
- j'aimerai connaitre la [qualité de l'air](topic) à Paris
- Je voudrai avoir [les chiffres](object) de la [population française](topic)
- quel est [mon numéro](object) [RNA](topic) ?
- poule 2010
- je veux des données
- [vigicrue](topic) 2018
- base [SIRENE](topic)

## intent:confirm
- oui
- ouais
- ouaip
- ui
- hum oui
- hmm ui
- ok
- c'est ça
- yes

## intent:deny
- non
- pas du tout
- nope
- oui mais non
- hm non
- hum non
- no

## intent:thankyou
- merci
- c'est chouette
- cool !
- super !

## intent:bye
- au revoir
- à plus
- bye
- ciao
- bonne nuit

## lookup:organization
organisations.txt

## lookup:topic
datasets.txt

## regex:year
- [0-9]{4}
"""
%store nlu_md > nlu.md

## importation des noms d'organisations

In [ ]:
import pandas as pd

In [ ]:
df1 = pd.read_csv('../pilotage/data/organizations.csv', sep=";")

In [ ]:
names = [ x for x in list(df1['name'].astype(str).values) if x != 'nan' ]
accronyms = [ x for x in list(df1['acronym'].astype(str).values) if x != 'nan' ]

txt = "\n".join(names + accronyms)
%store txt > organisations.txt

In [ ]:
df2 = pd.read_csv('../pilotage/data/datasets.csv', sep=";")

In [ ]:
df2.head()

names = [ x for x in list(df2['title'].astype(str).values) if x != 'nan' ]
accronyms = [ x for x in list(df2['acronym'].astype(str).values) if x != 'nan' ]

txt = "\n".join(names + accronyms)
%store txt > datasets.txt

## Entrainement

In [ ]:
!python -m rasa_nlu.train -c nlu_config.yml --data nlu.md -o models --fixed_model_name nlu --project current --verbose

In [ ]:
interpreter = Interpreter.load("./models/current/nlu")

In [ ]:
message = "Pays de Morlaix 2017"
result = interpreter.parse(message)
print(json.dumps(result, indent=2))

In [ ]:
message = "oui"
result = interpreter.parse(message)
print(json.dumps(result, indent=2))

In [ ]:
message = "non"
result = interpreter.parse(message)
print(json.dumps(result, indent=2))